In [415]:
import numpy as np
import typing

In [416]:
DTYPE = np.float64
np.set_printoptions(
    precision=4,
    linewidth=100,
    suppress=True,
)


In [ ]:
np.random.seed(52)

rng = np.random.default_rng()

N = rng.integers(low=3, high=7)
L = rng.integers(low=2, high=5)
while L > N:
    L = rng.integers(low=2, high=5)

print(N, L)


3 4


In [418]:
# 2 на 10, 2 на 100
def gen_simple_matrix(N: int, L: int) -> np.ndarray:
    P = np.zeros((N, L), dtype=DTYPE)
    for i in range(L):
        random_values = rng.uniform(low=-10.0, high=10.0, size=N - i).astype(DTYPE)
        while np.any(random_values == 0.0):
            random_values = rng.uniform(low=-10.0, high=10.0, size=N - i).astype(DTYPE)
        P[0 : N - i, i] = random_values
    print(P, P.dtype)
    return P


gen_simple_matrix(N, L)


[[ 0.2757  2.8798  6.1473  0.    ]
 [ 3.3661  6.3505  0.      0.    ]
 [-1.2143  0.      0.      0.    ]] float64


array([[ 0.2757,  2.8798,  6.1473,  0.    ],
       [ 3.3661,  6.3505,  0.    ,  0.    ],
       [-1.2143,  0.    ,  0.    ,  0.    ]])

In [419]:
# 2 на 10, 2 на 100
def gen_well_conditioned(N: int, L: int) -> np.ndarray:
  P = gen_simple_matrix(N, L)
  P[:, 0] = P[:, 0] + 100
  print(P, P.dtype)
  return P

gen_well_conditioned(N, L)

[[-1.2444 -5.0328  7.3114  0.    ]
 [-3.4427 -7.8973  0.      0.    ]
 [ 5.9995  0.      0.      0.    ]] float64
[[ 98.7556  -5.0328   7.3114   0.    ]
 [ 96.5573  -7.8973   0.       0.    ]
 [105.9995   0.       0.       0.    ]] float64


array([[ 98.7556,  -5.0328,   7.3114,   0.    ],
       [ 96.5573,  -7.8973,   0.    ,   0.    ],
       [105.9995,   0.    ,   0.    ,   0.    ]])

In [420]:
# по две размерности на каждый k
def gen_ill_conditioned(N: int, L: int, k: int) -> np.ndarray:
    P = gen_simple_matrix(N, L)

    P[:, 0] *= 10 ** (-k)

    print(P)
    return P

gen_ill_conditioned(N, L, 2)

[[-4.7026 -1.6787  0.9776  0.    ]
 [-4.178   5.1003  0.      0.    ]
 [-7.2626  0.      0.      0.    ]] float64
[[-0.047  -1.6787  0.9776  0.    ]
 [-0.0418  5.1003  0.      0.    ]
 [-0.0726  0.      0.      0.    ]]


array([[-0.047 , -1.6787,  0.9776,  0.    ],
       [-0.0418,  5.1003,  0.    ,  0.    ],
       [-0.0726,  0.    ,  0.    ,  0.    ]])

In [421]:
def gen_RHS(N: int) -> np.ndarray:
  return rng.uniform(0, 1, size=(N,))

gen_RHS(N)

array([0.7207, 0.9242, 0.5911])

In [422]:
# i* = i
# j* = j - i + L


def K0(i: int, N: int, L: int) -> int:
    return 0 if i + 1 <= L else i + 1 - L + 1


def KN(i: int, N: int, L: int) -> int:
    return i + 1 + L - 1 if i + 1 <= N - L else N


def cholesky_LU(A: np.ndarray):
    N = A.shape[0]
    L = A.shape[1]

    B = np.zeros(shape=(N, N), dtype=DTYPE)
    C = np.eye(N, dtype=DTYPE)

    # print(B)

    for j in range(N):
        for i in range(j, N):
            # print(f"i, j = ({i} {j})")
            if 0 <= np.abs(j - i) <= L - 1:
                B[i, j] += A[i, j - i] if i <= j else A[j, i - j]

            summation = 0
            for k in range(j):
              summation += B[i, k] * C[k, j]
            
            # summation = np.sum(B[i, 0 : j - 1] * C[0 : j - 1, j])

            # print(summation)

            B[i, j] -= summation
        # print(B)


        for jj in range(j + 1, N):
            if 0 <= np.abs(jj - j) <= L - 1:
                C[j, jj] += A[j, jj - j] if j <= jj else A[jj, j - jj]

            summation = 0
            for k in range(j):
              summation += B[j, k] * C[k, jj]

            # print(summation)
            C[j, jj] -= summation

            # summation = np.sum(B[j, 0 : j - 1] * C[0 : j - 1, jj])

            C[j, jj] /= B[j, j]

        # print(B)
    # print(B)
    return B, C

def cholesky_LU_symm(A: np.ndarray):
    N = A.shape[0]
    L = A.shape[1]

    B = np.zeros(shape=(N, N), dtype=DTYPE)

    for j in range(N):
        for i in range(j, N):
            # print(f"i, j = ({i} {j})")
            if 0 <= np.abs(j - i) <= L - 1:
                B[i, j] += A[i, j - i] if i <= j else A[j, i - j]

            summation = 0
            for k in range(j):
                # print(B[i, k], B[j, k])
                # print(B[k, k])
                summation += B[i, k] * B[j, k]
                summation /= B[k, k]
            
            # print(summation)

            B[i, j] -= summation

    return B

def cholesky_LU_symm_find_y(B: np.ndarray, f: np.ndarray):
    N = B.shape[0]

    y = np.zeros(N)

    for i in range(N):
      y[i] = f[i]
      summation = 0
      for k in range(i):
        summation += B[i, k]*y[k]
      y[i] -= summation
      y[i] /= B[i, i]

    return y

def cholesky_LU_symm_find_x(B: np.ndarray, y: np.ndarray):
    N = B.shape[0]

    x = np.zeros(N)

    for i in range(N - 1, -1, -1):
        x[i] = y[i]
        summation = 0
        for k in range(i + 1, N):
            summation += B[k, i] * x[k]
        summation /= B[i, i]

        x[i] -= summation
    return x
  

def cholesky_LU_backward():
  pass

def cholesky_forward(A: np.ndarray, f: np.ndarray):
    N = A.shape[0]
    L = A.shape[1]

    # By = f
    B = np.zeros(shape=(N, N), dtype=DTYPE)
    for i in range(0, N):
        for j in range(i, N):
            print(i, j)
            if 0 <= np.abs(j - i) <= L - 1:
                B[i, j] += A[i, j - i] if i <= j else A[j, i - j]
                # print(B[i, j])

            print(B)

            sum = 0
            for k in range(K0(i, N, L), j):
                print(k)
                part_sum = B[i, k] * B[j, k]
                part_sum /= B[k, k]
                print(f"part_sum: {part_sum}")
                sum += part_sum
            B[i, j] -= sum

    print(B)
    return B


# cholesky_forward(gen_simple_matrix(N, L), gen_RHS(N))
A = gen_simple_matrix(N, L)
B, C = cholesky_LU(A)
print("B: ")
print(B)
print("C: ")
print(C)

sym_f = gen_RHS(N)
sym_B = cholesky_LU_symm(A)
print(sym_B)
sym_y  = cholesky_LU_symm_find_y(sym_B, sym_f)
print(sym_y)
print(np.matmul(sym_B, sym_y))
print(sym_f)

sym_x = cholesky_LU_symm_find_x(sym_B, sym_y)
print(sym_x)


[[-9.9877  5.6384  8.6289  0.    ]
 [ 8.2284 -1.5803  0.      0.    ]
 [-3.5297  0.      0.      0.    ]] float64
B: 
[[-9.9877  0.      0.    ]
 [ 5.6384 11.4115  0.    ]
 [ 8.6289  3.291   2.9761]]
C: 
[[ 1.     -0.5645 -0.8639]
 [ 0.      1.      0.2884]
 [ 0.      0.      1.    ]]
[[-9.9877  0.      0.    ]
 [ 5.6384 11.4115  0.    ]
 [ 8.6289  3.291  -3.8255]]
[-0.0815  0.0491 -0.3357]
[0.814  0.1013 0.7427]
[0.814  0.1013 0.7427]
[-0.2891  0.146  -0.3357]
